In [2]:
%load_ext tensorboard

In [3]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [5]:

with open('Reddit_Blonde.txt', 'r') as file:
    text = file.read().replace('\n', '')

chars = sorted(list(set(text)))

for c in chars:
    if not(44 <= ord(c) <= 57 or 63 <= ord(c) <= 90 or 97 <= ord(c) <= 122 or c == ' ' or c == '!') :
        text = text.replace(c, '')

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])
    
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1

In [6]:
print(text[:200])

A middle aged frumpy married couple return to a Mercedes dealership where the salesman has just sold the car they were interested in to a beautiful leggy busty blonde...  I thought you said you would 


In [8]:

model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.02),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 128)           101376    
_________________________________________________________________
dense_1 (Dense)              (None, 40, 69)            8901      
Total params: 110,277
Trainable params: 110,277
Non-trainable params: 0
_________________________________________________________________


In [7]:
def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

    generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)

            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [13]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  
    preds = preds / np.sum(preds)                
    probas = np.random.multinomial(1, preds, 1)  
    return np.argmax(probas)                     

In [37]:
model.fit(x, y,
          batch_size=128,
          epochs=100,
          callbacks=[print_callback])

Epoch 1/100
211/212 [============================>.] - ETA: 0s - loss: 1.2221 - accuracy: 0.6369
----- Generating text after Epoch: 0
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: "finishes to talk, St. Peters snaps his f"
finishes to talk, St. Peters snaps his frenvines by a sGreet at a car to license to. The Blonde immersians at the momen, and passes back so we dont servitls. @@@ Did you have her new schose?Blonde walks up to the counter.     @@@ blonde, poiduats into the weled the blonde knows to make shades.What it is.He remens fine says, Thanks, after the blonde, and progressive will know, Im a blond did it been computer in a game were driving back o
212/212 [==============================] - 22s 106ms/step - loss: 1.2222 - accuracy: 0.6369
Epoch 2/100
211/212 [============================>.] - ETA: 0s - loss: 1.2183 - accuracy: 0.6380
----- Generating text after Epoch: 1
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
--

s a voice over the radio saying, This is put given barka owner the car fout your pervert of Buy! Unfazy mother. Then you mermb.At the potato. The train calls a sure, a sele, but the blonde believe her becural me counts this undergroup how about thispend in the 99th she is the store... Because the ledge opened it.Blonde came! pool! @@@ Why did her on building. He opens her 90 biropens. As the blonde jump against who has 2 boy? @@@ Blonde
212/212 [==============================] - 25s 116ms/step - loss: 1.1961 - accuracy: 0.6441
Epoch 12/100
211/212 [============================>.] - ETA: 0s - loss: 1.1952 - accuracy: 0.6443
----- Generating text after Epoch: 11
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: "he doctor... Doctor What seems to be tro"
he doctor... Doctor What seems to be troes to the alligator. She rushes their hands the money.Youre at you I lamp you hair? A blonde jump. Its I asked, Whater took the cop replies, It said Already 

 up to our heads!And the blonde replies, you didnt kind her. The lawyer all the Deviled.. in wighing a drive rect sell blonde have blondes So the brunette. Brunette I didnt keeper, eassy ladied, I turns to him. Blondestatches the shop shop and she walks up to a blonde?a ppelstanothlays the offity.The pying his blonde woman goes home a ravelyn with a crowding @@@ A fele@.. of the air traclians, at you, awres how many difference buttont w
212/212 [==============================] - 28s 132ms/step - loss: 1.1745 - accuracy: 0.6501
Epoch 32/100
211/212 [============================>.] - ETA: 0s - loss: 1.1738 - accuracy: 0.6498
----- Generating text after Epoch: 31
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: "m in after a short back and forth the ha"
m in after a short back and forth the hand cans it. She replied purse, ien the marks and sniff stoused to the train runs In Way, how more hot the head opening. @@@ Why bosy a redhead, a brunette, a

 towards the captain who tells her, that bosing, she decides lemonles! @@@ A few minutes, and sure out the door on the mideft of sureper A Blonde yells? She rub again.warcig on itranes and concerner to meoph with four longers! @@@ A blonde walks into a staying must black and goes to her jokes and mother asks if she has turring the other side? BAF magacuess. @@@ Blonde boance in first black. She breds there. The lawyer, What do going to 
212/212 [==============================] - 44s 207ms/step - loss: 1.1631 - accuracy: 0.6529
Epoch 52/100
211/212 [============================>.] - ETA: 0s - loss: 1.1626 - accuracy: 0.6526
----- Generating text after Epoch: 51
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: "is finally happy.The farmer says how did"
is finally happy.The farmer says how did you figal! Maam about it, it ishis, oh tooooo, was your head,edower and says, Yes, we do the woman friend at tither. Hellow thing. A very arry around a fire

l the other kids could only count to found. @@@ Ferrinataly and come bucked all of the long wifeSoured another bridge but another trouced room and baligh, noticers. Which darence and says, I stops up, so a blonde girl duranss scrip nice. The blonde replies in my head. Skeventicated. Not waiting blusing tells the 3rd jokes and animal her piror?Laur? I like it. A Blonde So, I didnt want to put shooked, very to tollige tailine The guns are
212/212 [==============================] - 44s 208ms/step - loss: 1.1541 - accuracy: 0.6551
Epoch 72/100
211/212 [============================>.] - ETA: 0s - loss: 1.1536 - accuracy: 0.6552
----- Generating text after Epoch: 71
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: "he heard her right exclaims, but maam th"
he heard her right exclaims, but maam the probable husband, sand walking to your proclade says This gazed only unlany, says it dont know the sama time... I havential. This readures My family With b

wife, inside the house, heard the conversatherest down, Redhead basist. @@@ How did to going a blonde? The blonde is going to rocks you with send a dog and a blonde calls his home... Why, whats the night. Two blonde, a redhead, and8 and a blonde potatoessted goes on a fire. @@@ While blondes move to follow I cant ans jussed back and home by a blonde with two beazing. @@@ A blonde gets from Laid A blonde pulls out the sore city? You excl
212/212 [==============================] - 45s 210ms/step - loss: 1.1477 - accuracy: 0.6569
Epoch 92/100
211/212 [============================>.] - ETA: 0s - loss: 1.1472 - accuracy: 0.6569
----- Generating text after Epoch: 91
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: " the redhead and stood them against the "
 the redhead and stood them against the guy. Why are you desprighe to mpecting for side of extra, smile gum then She turns around found says, Giveould to the he handsabo went at the py? Theyre a bl

In [35]:
def generate(length):

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        #sentence = seed
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(length):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)

            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [38]:
generate(200) 

----- diversity: 0.2
----- Generating with seed: "e, and a redhead are in the ninth grade."
e, and a redhead are in the ninth grade. The blonde replied, I think you can go to the blonde and she says I was going to be able to get a shower and says I will sho test another blonde and a brunette are all the store and says I was down t
----- diversity: 0.5
----- Generating with seed: "e, and a redhead are in the ninth grade."
e, and a redhead are in the ninth grade. Here it is pluce to the bull. You tell the two blonde, I will sho that first class show back in home. Finally, the police was down the bar he were an yell. The blonde says, How did you make a telegra
----- diversity: 1.0
----- Generating with seed: "e, and a redhead are in the ninth grade."
e, and a redhead are in the ninth grade.When I follought arrive in the Abilot ever enough antiluld sisterday, why is the high home.  . Same it going to get paying forward through the QUlof with Next the Propors, smile please it is at the an


In [21]:
on_epoch_end(1, _)


----- Generating text after Epoch: 1
----- diversity: 0.2
----- diversity: 0.5
----- diversity: 1.0
----- Generating with seed: "redhead and a blonde were to be executed"
redhead and a blonde were to be executed over to the barnthing for her horm alreader, and then a box as wet.Ouch, impresserb.  The Frenchman left at the ranct thats up as her first thing for where stran colks up to the understand again.Fice. She said if I check how blonde shop and spages on the baryexed at the brunette turns. The blonde is driving in the other girl before the two  You sell to blondes.You dont know the dog dead off the t
